<a href="https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/object_detection_for_image_cropping_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO in Darkflow to detect birds from images
---
*Last Updated 9 December 2019*   
Using YOLO via Darkflow as a method to do customized, large-scale image processing. Using the location and dimensions of the detected birds, images will be cropped to square dimensions that are centered and padded around the detection box. Pre-trained models are used for "out of the box" inference on images of birds of varying dimensions and resolutions, but will be modified and fine-tuned in future efforts for other taxonomic groups.

This notebook is meant to be run enitrely in Google Colab and doesn't require any software installations or downloads to your local machine. To get started, just click the "Open in Colab" button. 

Note: Darkflow is currently only compatible with YOLO v1 and v2.

## Installs
---
Install required libraries directly to this Colab notebook.

In [0]:
# Make sure you are using Python 3.6
# Install packages using pip
!python --version
!pip install tensorflow-gpu==1.15.0rc2
!pip install cython
!pip install opencv-python

In [0]:
# Download darkflow (the tensorflow implementation of YOLO)
import os
import pathlib


if "darkflow-master" in pathlib.Path.cwd().parts:
  while "darkflow-master" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path("darkflow-master").exists():
  !git clone --depth 1 https://github.com/thtrieu/darkflow.git

In [0]:
# Compile darkflow
%cd darkflow
!python setup.py build_ext --inplace

In [0]:
# Change darkflow to darkflow-master to distinguish between folder names
%cd ../
!mv darkflow darkflow-master

In [0]:
# (Optional) Mount google drive to export detection results as tsv
# You can also test everything without running this chunk, detection results just won't be exported
from google.colab import drive
drive.mount('/content/drive')

### Imports   
---

In [0]:
%cd darkflow-master

%tensorflow_version 1.15.0rc2
import tensorflow as tf
print(tf.__version__)

# For importing/exporting files, working with arrays, etc
import pathlib
import time
import csv
import urllib
import numpy as np
import pandas as pd

# For the actual object detection
from darkflow.net.build import TFNet

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
import cv2
%config InlineBackend.figure_format = 'svg'

### Model Preparation
---   
**Uploads**: The models are already in darkflow/cfg, but the pre-trained weights associated with these models need to be uploaded to this notebook from https://drive.google.com/drive/folders/0B1tW_VtY7onidEwyQ2FtQVplWEU. 

**"Flowing" images through the model**: Ignore the warning messages about deprecated names, they still work at the time this last updated. Code for parameters is based on https://github.com/thtrieu/darkflow ("Using darkflow from another python application").

Your output should be a table of values like those shown below:

Source | Train? | Layer description                | Output size
------- |:--------:|:----------------------------------:| ---------------
       |        | input                            | (?, 448, 448, 3)
 Load  |  Yep!  | scale to (-1, 1)                 | (?, 448, 448, 3)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 448, 448, 16)

**Define boxing function**: You can adjust the parameters so that bounding boxes are only shown for certain confidence or class values. Here boxes are shown when confidence > 0.45 and object class is 'bird'. This function is modified from here https://gist.github.com/deep-diver/40f092ad56525189674a86b6fde6d304.

In [0]:
# Upload yolo.weights, pre-trained weights file (for YOLO v2) from Google drive 
# For directions to upload other weights files, see the wiki for this repository
weights = 'yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'):
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

In [0]:
# Upload yolo-tiny.weights
weights = 'yolo-tiny'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'):
  !gdown --id 0B1tW_VtY7onibmdQWE1zVERxcjQ
  !mv yolo-tiny.weights bin

In [0]:
# Move yolo-tiny.cfg model from cfg/v1 to cfg/
# File from https://github.com/cvjena/darknet/blob/master/cfg/yolo-tiny.cfg
!mv cfg/v1/yolo-tiny.cfg cfg

In [0]:
# Test installation, you should see an output with different parameters for flow
%cd darkflow-master
!pwd
!python flow --h

In [0]:
# Define parameters for "flow"ing the images through the model
# Can change model and weights files here, tiny-yolo is faster but less accurate
params = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.45, 
    'gpu': 1.0
}

# Run the model
tfnet = TFNet(params)

In [0]:
# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

In [0]:
# For drawing bounding boxes around detected objects on images
def boxing(image, predictions):
    newImage = np.copy(image)
    im_height, im_width, im_depth = image.shape
        
    for result in predictions:
        xmin = result['topleft']['x']
        ymin = result['topleft']['y']

        xmax = result['bottomright']['x']
        ymax = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        # only show boxes that are above .1 confidence and for the label, bird
        if confidence > 0.45 and result['label'] == 'bird' :
            # draw boxes on images
            fontScale = min(im_width,im_height)/(600)
            newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
            newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

            # optional: if mounted to drive, export detection results to sample_crops_yolo.tsv
            # must make empty sample_crops_yolo.tsv file in your drive first and paste the path to it below
            if os.path.exists('/content/drive/My Drive/fall19_smithsonian_informatics/sample_crops_yolo_1.tsv'):
              with open('/content/drive/My Drive/fall19_smithsonian_informatics/sample_crops_yolo_20000imgc.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  #crop_width = xmax-xmin
                  #crop_height = ymax-ymin
                  tsv_writer.writerow([image_url, im_height, im_width, 
                            xmin, ymin, xmax, ymax])
            
        else:
          print("No birds detected in {}.".format(image_url))
    return newImage

## Load in sample images and 'flow' them through the object detector
---
You can either **A) Load individual images in by URL**, or for large image batches or **B) Load multiple images from a text file of image URLs**. Other methods for importing to Google Colab are listed [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd). 

**A) Load individual images in by URL**
Load in images by URL and run the image detector for all images. Plotted results include the image with bounding box around detected objects (birds), class type, and confidence score. Inference times are printed above images. If you "mounted" your Google Drive during "Installs", the bounding box coordinates will also be written to 'sample_crops_yolo.tsv'.

In [0]:
image_urls = ["https://content.eol.org/data/media/7e/9c/7a/542.15445377044.jpg",
              "https://content.eol.org/data/media/81/1c/0d/542.7816025222.jpg",
              "https://content.eol.org/data/media/7e/3c/0b/542.10578857864.jpg"]

In [0]:
for image_url in image_urls:
  image = url_to_image(image_url)

  # Use YOLO for object detection  
  # Record inference time
  start_time = time.time()
  result = tfnet.return_predict(image)
  end_time = time.time()

  # Plot and show detection boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(boxing(image, result))

  # Display inference time above images
  plt.title('Inference time: {}'.format(format(end_time-start_time, '.2f')))

**B) Load multiple images from a text file of image URLs**
Load in multiple images from a text file of URLS and run the image detector for all images. Plotted results include the image with bounding box around detected objects (birds), class type, and confidence score. Inference times are printed above images. If you "mounted" your Google Drive during "Installs", the bounding box coordinates will also be written to 'sample_crops_yolo.tsv'.

In [0]:
urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Aves_20K_breakdown_download_000001.txt'
df1 = pd.read_csv(urls)
df1.columns = ["link"]
pd.DataFrame.head(df1)

In [0]:
# Loops through first 5 image urls from the text file
for i, row in df1.head(5).itertuples(index=True, name='Pandas'):

# For ranges of rows or all rows, use the commands below
# Can be useful if running batch jobs
#for i, row in df1.iloc[500:800].iterrows():
#for i, row in df1.itertuples(index=True, name='Pandas'):
#for i, row in df1.tail(5).itertuples(index=True, name='Pandas'):

  try:
    # Record inference time
    start_time = time.time()
    image_url = df1.get_value(i, "link")
    image = url_to_image(image_url)
    # Detection
    result = tfnet.return_predict(image)
    end_time = time.time()
    # Draw boxes on images
    boxing(image, result)
  
    # If running detection on >50 images, do not display detection results
    # Instead run below command to track progress
    #print('Detection complete in {} of 1,000 images'.format(i+1))
  
  except:
    print('Error: check if web address is valid')
  
  # Plot and show detection boxes on images
  # Hashtag out this portion if running detection on >50 images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(boxing(image, result))
  plt.title('{}) Inference time: {}'.format(i+1, format(end_time-start_time, '.2f')))
  plt.close()